#### Downloading PureGym Youtube Videos Metadata

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import yt_dlp
import pandas as pd

# Playlist URL
playlist_url = 'https://youtube.com/playlist?list=PLsliPlpqH8JD9Iy1A3WiiSEGPMvpX3voq&feature=shared'

# yt-dlp options
ydl_opts = {
    'extract_flat': True,  # Only extract video metadata
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    playlist_info = ydl.extract_info(playlist_url, download=False)

# Collect video titles, URLs, and thumbnail URLs
videos = [
    {
        'Title': video['title'],
        'URL': f"https://www.youtube.com/watch?v={video['id']}",
        'Thumbnail_URL': video.get('thumbnails', [{}])[-1].get('url', 'No thumbnail')
    }
    for video in playlist_info['entries']
]

# Create DataFrame
yt_data = pd.DataFrame(videos)

# Display and save DataFrame
print(yt_data.head())


[youtube:tab] Extracting URL: https://youtube.com/playlist?list=PLsliPlpqH8JD9Iy1A3WiiSEGPMvpX3voq&feature=shared
[youtube:tab] PLsliPlpqH8JD9Iy1A3WiiSEGPMvpX3voq: Downloading webpage
[youtube:tab] PLsliPlpqH8JD9Iy1A3WiiSEGPMvpX3voq: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: EXERCISE DEMONSTRATIONS
[youtube:tab] PLsliPlpqH8JD9Iy1A3WiiSEGPMvpX3voq page 1: Downloading API JSON
[youtube:tab] PLsliPlpqH8JD9Iy1A3WiiSEGPMvpX3voq page 2: Downloading API JSON
[youtube:tab] Playlist EXERCISE DEMONSTRATIONS: Downloading 266 items of 266
[download] Downloading item 1 of 266
[download] Downloading item 2 of 266
[download] Downloading item 3 of 266
[download] Downloading item 4 of 266
[download] Downloading item 5 of 266
[download] Downloading item 6 of 266
[download] Downloading item 7 of 266
[download] Downloading item 8 of 266
[download] Downloading item 9 of 266
[download] Downloading item 10 of 266
[download] Downloading item 11 of 266
[download] 

[download] Downloading item 199 of 266
[download] Downloading item 200 of 266
[download] Downloading item 201 of 266
[download] Downloading item 202 of 266
[download] Downloading item 203 of 266
[download] Downloading item 204 of 266
[download] Downloading item 205 of 266
[download] Downloading item 206 of 266
[download] Downloading item 207 of 266
[download] Downloading item 208 of 266
[download] Downloading item 209 of 266
[download] Downloading item 210 of 266
[download] Downloading item 211 of 266
[download] Downloading item 212 of 266
[download] Downloading item 213 of 266
[download] Downloading item 214 of 266
[download] Downloading item 215 of 266
[download] Downloading item 216 of 266
[download] Downloading item 217 of 266
[download] Downloading item 218 of 266
[download] Downloading item 219 of 266
[download] Downloading item 220 of 266
[download] Downloading item 221 of 266
[download] Downloading item 222 of 266
[download] Downloading item 223 of 266
[download] Downloading it

In [3]:
## Cleaning
yt_data.rename(columns={'Title': 'Exercise_Name'}, inplace=True)
yt_data['Exercise_Name'] = yt_data['Exercise_Name'].str.replace("How To Do A", '', regex=False).str.strip()
yt_data['Exercise_Name'] = yt_data['Exercise_Name'].str.replace("How To Do", '', regex=False).str.strip()

In [4]:
yt_data.head()

,Exercise_Name,URL,Thumbnail_URL
0,Plank To Push Up,https://www.youtube.com/watch?v=56vUOad6Irs,https://i.ytimg.com/vi/56vUOad6Irs/hqdefault.j...
1,V Sit Up,https://www.youtube.com/watch?v=cSCyyfBti7U,https://i.ytimg.com/vi/cSCyyfBti7U/hqdefault.j...
2,Pulse Sit Ups,https://www.youtube.com/watch?v=O8i7KeZqP9A,https://i.ytimg.com/vi/O8i7KeZqP9A/hqdefault.j...
3,Swiss Ball Pikes,https://www.youtube.com/watch?v=vo3csqN6rpo,https://i.ytimg.com/vi/vo3csqN6rpo/hqdefault.j...
4,Sit Ups,https://www.youtube.com/watch?v=T3XsCC2Td1g,https://i.ytimg.com/vi/T3XsCC2Td1g/hqdefault.j...


In [5]:
yt_data.to_csv('data/youtube_playlist_urls_and_thumbnails.csv',index=False)


#### Getting API Keys and Models

In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
HF_API_KEY=os.getenv("HF_API_KEY")

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm_model = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=GOOGLE_API_KEY,temperature=0)

#### Loading, Splitting, and Embedding and Storing in VectorDB The Latest Health Report

In [8]:
#Loading Report
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/Health Report- Yash Sakhuja.pdf")
pages = loader.load()
print(len(pages))

14


In [9]:
#Chunking & Splitting the document
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

In [10]:
splits= text_splitter.split_documents(pages)
len(splits)

31

In [11]:
#Embedding the Document using HF-SentenceTransformer Model
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HF_API_KEY, model_name="sentence-transformers/all-MiniLM-l6-v2"
)

In [16]:
#Storing in VectorDB (ChromaDB)
from langchain.vectorstores import FAISS
vectordb = FAISS.from_documents(splits, embedding=embeddings)

In [ ]:
#vectordb.delete_collection()

#### Importing & Processing PureGym Data

In [17]:
import pandas as pd
data=pd.read_csv("data/youtube_playlist_urls_and_thumbnails.csv")

In [18]:
data

,Exercise_Name,URL,Thumbnail_URL
0,Plank To Push Up,https://www.youtube.com/watch?v=56vUOad6Irs,https://i.ytimg.com/vi/56vUOad6Irs/hqdefault.j...
1,V Sit Up,https://www.youtube.com/watch?v=cSCyyfBti7U,https://i.ytimg.com/vi/cSCyyfBti7U/hqdefault.j...
2,Pulse Sit Ups,https://www.youtube.com/watch?v=O8i7KeZqP9A,https://i.ytimg.com/vi/O8i7KeZqP9A/hqdefault.j...
3,Swiss Ball Pikes,https://www.youtube.com/watch?v=vo3csqN6rpo,https://i.ytimg.com/vi/vo3csqN6rpo/hqdefault.j...
4,Sit Ups,https://www.youtube.com/watch?v=T3XsCC2Td1g,https://i.ytimg.com/vi/T3XsCC2Td1g/hqdefault.j...
...,...,...,...
261,Hang Power Snatch Complex,https://www.youtube.com/watch?v=sTVLg3zq_WY,https://i.ytimg.com/vi/sTVLg3zq_WY/hqdefault.j...
262,Dumbbell Squat Cleans,https://www.youtube.com/watch?v=chES6BtEBVU,https://i.ytimg.com/vi/chES6BtEBVU/hqdefault.j...
263,Dumbbell Cleans,https://www.youtube.com/watch?v=5xJjJzdO-oE,https://i.ytimg.com/vi/5xJjJzdO-oE/hqdefault.j...
264,Dumbbell Snatch,https://www.youtube.com/watch?v=gY7awqQmWlg,https://i.ytimg.com/vi/gY7awqQmWlg/hqdefault.j...


In [19]:
ex_names_list = ",".join(data['Exercise_Name'].astype(str).tolist())

print(ex_names_list)

Plank To Push Up,V Sit Up,Pulse Sit Ups,Swiss Ball Pikes,Sit Ups,Swiss Ball Circles,Side Plank,Plank,Ab Exercise - How to plank knee to elbow,How To Plank From Knee (Modified Plank),Single Arm Curl (Bicep Curl),n Overhead Tricep Extension,Skull Crushers,Dumbbell Shoulder Press,How To Tricep Press Up,How To Press Up From Knees (Modified Press Up),uble Arm Curl (Dumbbell Bicep Curl),Kettlebell Swing,Mountain Climbers,Burpees,Forward Lunge,Glute Bridge,How To Reverse Lunge,Jumping Lunges,Barbell front squat,Goblet squat,How To Jump Squat,Broad Jumps,Medicine Ball Slams,Leg exercise -  Romanian Deadlift,Press Ups,High Knees,Sumo Squat,Barbell Back Squat,Hack Squats On The Hack Squat Machine,Split Squat,LEG EXERCISE  - How to do the clock lunge,Lateral Lunges (Side Lunges),Curtsy Lunge,Walking Lunges,Sprinter Lunges,Half Burpee (A Down Up),The Arnold Press,The Military Press,Shoulder Press,Cable Curl,Concentration Curls,Hammer Curls,Preacher Curls,Spider Curls,Zottman Curl,How To  Do A Sing

In [20]:
exercise_names_list = ex_names_list.split(",")  # Splitting on double spaces

# Specify the directory and filename
directory = "data"
filename = "exercise_names.txt"
file_path = os.path.join(directory, filename)

# Create the directory if it doesn't exist
os.makedirs(directory, exist_ok=True)

# Open the file in write mode within the specified directory
with open(file_path, 'w') as file:
    # Write each exercise name on a new line
    for exercise in exercise_names_list:
        # Remove any leading/trailing whitespace and write to file
        file.write(f"{exercise.strip()}\n")

print(f"Data has been written to {file_path}.")

Data has been written to data/exercise_names.txt.


#### Querying & Parsing

In [21]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Define your desired data structure.
class Parsing(BaseModel):
    day_of_week: str = Field(description="Give me day of the week the exercise is for.")
    exercise_name: str = Field(description="Give me the exercise name from the list.")
    sets_reps: str = Field(description="Give me the sets and reps I have to do that exercise for.")
    rest: str = Field(description="Give me the rest I have to take between consecutive reps.")

parser = JsonOutputParser(pydantic_object=Parsing)

/usr/local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3397: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [22]:
from langchain.prompts import PromptTemplate


template= """ Assume the role of my gym trainer who would go through my health checkup report and help me design a training plan
to help me improve my physical health and keep me free from cardiac problems all my life.

I am 23 year old male who would like to hit the gym for 1 hour a day and my purpose is to maintain a good heart condition and 
reduce belly fat and build muscle mass without any protien intake from outside sources other than healthy balance diet.

Use My latest health reports for context and understanding my vitals and body condition.
{context}

Only give exercise names from the list of exercises mentioned below. Do not give any other exercise name. Make Sure you give the names with same exact spellings as mentioned in list.
\n{ex_names}\n

Make sure you maintain this regime
Monday: Cardio & Legs
Wednesday: Chest & Triceps
Thursday: Core
Friday: Back & Biceps
Saturday: Shoulders & Core


I start all my Gym sessions with a cycling of 30 minutes. Don't metnion cycling in the list you provide just give me the list of exercises after cycling.

So all exercises or weight training should begin after that and I do not want to spend only 30 minutes (exclusding cycling time).DS_Store
Do not give more than 5 exercises in a day.

\n{format_instructions}\n

Question: {question}
Helpful Answer:  """



QA_CHAIN_PROMPT = PromptTemplate.from_template(template,partial_variables={"format_instructions": parser.get_format_instructions()})

In [23]:
QA_CHAIN_PROMPT

PromptTemplate(input_variables=['context', 'ex_names', 'question'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"day_of_week": {"title": "Day Of Week", "description": "Give me day of the week the exercise is for.", "type": "string"}, "exercise_name": {"title": "Exercise Name", "description": "Give me the exercise name from the list.", "type": "string"}, "sets_reps": {"title": "Sets Reps", "description": "Give me the sets and reps I have to do that exercise for.", "type": "string"}, "rest": {"title": "Rest"

In [24]:
chain =  QA_CHAIN_PROMPT | llm_model | parser

In [25]:
question="Suggest me a list of exercises in a day if visit the Gym 5 days a week with 2 rest days on Tuesday and Sunday."
response= chain.invoke({"context":vectordb.as_retriever(),"ex_names":ex_names_list,"question": question})

In [26]:
response

[{'day_of_week': 'Monday',
  'exercise_name': 'Barbell Back Squat',
  'sets_reps': '3 sets of 8-12 reps',
  'rest': '60 seconds'},
 {'day_of_week': 'Monday',
  'exercise_name': 'Leg exercise -  Romanian Deadlift',
  'sets_reps': '3 sets of 8-12 reps',
  'rest': '60 seconds'},
 {'day_of_week': 'Monday',
  'exercise_name': 'Forward Lunge',
  'sets_reps': '3 sets of 10-15 reps per leg',
  'rest': '60 seconds'},
 {'day_of_week': 'Monday',
  'exercise_name': 'Glute Bridge',
  'sets_reps': '3 sets of 15-20 reps',
  'rest': '60 seconds'},
 {'day_of_week': 'Monday',
  'exercise_name': 'Calf Raises',
  'sets_reps': '3 sets of 15-20 reps',
  'rest': '60 seconds'},
 {'day_of_week': 'Wednesday',
  'exercise_name': 'Barbell Bench Press Correctly',
  'sets_reps': '3 sets of 8-12 reps',
  'rest': '60 seconds'},
 {'day_of_week': 'Wednesday',
  'exercise_name': 'Dumbbell Bench Press',
  'sets_reps': '3 sets of 8-12 reps',
  'rest': '60 seconds'},
 {'day_of_week': 'Wednesday',
  'exercise_name': 'How To

In [27]:
df = pd.DataFrame(response)

In [28]:
df

,day_of_week,exercise_name,sets_reps,rest
0,Monday,Barbell Back Squat,3 sets of 8-12 reps,60 seconds
1,Monday,Leg exercise - Romanian Deadlift,3 sets of 8-12 reps,60 seconds
2,Monday,Forward Lunge,3 sets of 10-15 reps per leg,60 seconds
3,Monday,Glute Bridge,3 sets of 15-20 reps,60 seconds
4,Monday,Calf Raises,3 sets of 15-20 reps,60 seconds
5,Wednesday,Barbell Bench Press Correctly,3 sets of 8-12 reps,60 seconds
6,Wednesday,Dumbbell Bench Press,3 sets of 8-12 reps,60 seconds
7,Wednesday,How To Tricep Press Up,3 sets of as many reps as possible,60 seconds
8,Wednesday,n Overhead Tricep Extension,3 sets of 10-15 reps,60 seconds
9,Wednesday,Bodyweight Tricep Dip,3 sets of as many reps as possible,60 seconds


#### Final Data Cleaning

In [29]:
result = pd.merge(df, data, left_on='exercise_name',right_on="Exercise_Name", how='inner') 
result = result.drop(columns=['Exercise_Name'])
result = result.rename(columns={'exercise_name': 'exercise_name_list'})
result

,day_of_week,exercise_name_list,sets_reps,rest,URL,Thumbnail_URL
0,Monday,Barbell Back Squat,3 sets of 8-12 reps,60 seconds,https://www.youtube.com/watch?v=rrJIyZGlK8c,https://i.ytimg.com/vi/rrJIyZGlK8c/hqdefault.j...
1,Monday,Leg exercise - Romanian Deadlift,3 sets of 8-12 reps,60 seconds,https://www.youtube.com/watch?v=zU-f6DMCdAI,https://i.ytimg.com/vi/zU-f6DMCdAI/hqdefault.j...
2,Monday,Forward Lunge,3 sets of 10-15 reps per leg,60 seconds,https://www.youtube.com/watch?v=g8-Ge9S0aUw,https://i.ytimg.com/vi/g8-Ge9S0aUw/hqdefault.j...
3,Monday,Glute Bridge,3 sets of 15-20 reps,60 seconds,https://www.youtube.com/watch?v=tqp5XQPpTxY,https://i.ytimg.com/vi/tqp5XQPpTxY/hqdefault.j...
4,Monday,Calf Raises,3 sets of 15-20 reps,60 seconds,https://www.youtube.com/watch?v=Zep-wKHWkNM,https://i.ytimg.com/vi/Zep-wKHWkNM/hqdefault.j...
5,Wednesday,Barbell Bench Press Correctly,3 sets of 8-12 reps,60 seconds,https://www.youtube.com/watch?v=CjHIKDQ4RQo,https://i.ytimg.com/vi/CjHIKDQ4RQo/hqdefault.j...
6,Wednesday,Dumbbell Bench Press,3 sets of 8-12 reps,60 seconds,https://www.youtube.com/watch?v=AduT4Eq-iP0,https://i.ytimg.com/vi/AduT4Eq-iP0/hqdefault.j...
7,Wednesday,How To Tricep Press Up,3 sets of as many reps as possible,60 seconds,https://www.youtube.com/watch?v=VKEbp4SUKmg,https://i.ytimg.com/vi/VKEbp4SUKmg/hqdefault.j...
8,Wednesday,n Overhead Tricep Extension,3 sets of 10-15 reps,60 seconds,https://www.youtube.com/watch?v=9wxRhONFsRA,https://i.ytimg.com/vi/9wxRhONFsRA/hqdefault.j...
9,Wednesday,Bodyweight Tricep Dip,3 sets of as many reps as possible,60 seconds,https://www.youtube.com/watch?v=qrS6aa0aQ9I,https://i.ytimg.com/vi/qrS6aa0aQ9I/hqdefault.j...


In [30]:
## What's the day today
import datetime

# Get today's date
today = datetime.date.today()

# Get the day of the week (0 = Monday, 6 = Sunday)
day_of_week_today = today.strftime("%A")

print(day_of_week_today)

Friday


In [31]:
filtered_result=result[result["day_of_week"]==day_of_week_today]
filtered_result=filtered_result.reset_index(drop=True)
filtered_result

,day_of_week,exercise_name_list,sets_reps,rest,URL,Thumbnail_URL
0,Friday,Barbell Bent Over Row,3 sets of 8-12 reps,60 seconds,https://www.youtube.com/watch?v=FsK19s8eGFs,https://i.ytimg.com/vi/FsK19s8eGFs/hqdefault.j...
1,Friday,Pull Ups,3 sets of as many reps as possible,60 seconds,https://www.youtube.com/watch?v=PHdHnZcbsB8,https://i.ytimg.com/vi/PHdHnZcbsB8/hqdefault.j...
2,Friday,Reverse Grip Lat Pulldown,3 sets of 10-15 reps,60 seconds,https://www.youtube.com/watch?v=SNiwpA13ZLU,https://i.ytimg.com/vi/SNiwpA13ZLU/hqdefault.j...
3,Friday,Dumbbell Bent Over Row,3 sets of 8-12 reps per arm,60 seconds,https://www.youtube.com/watch?v=6gvmcqr226U,https://i.ytimg.com/vi/6gvmcqr226U/hqdefault.j...
4,Friday,uble Arm Curl (Dumbbell Bicep Curl),3 sets of 10-15 reps,60 seconds,https://www.youtube.com/watch?v=MtXdEcW3Eog,https://i.ytimg.com/vi/MtXdEcW3Eog/hqdefault.j...


In [34]:
output_template= """
For the following text, extract only the name of the exercise and nothing else.

Return as simple string and no escape sequences or markdown.

Return Null if not appropriate answer available. Do not hallucinate.

text: {text}

{format_instructions}

"""

from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser
from langchain.output_parsers import ResponseSchema
import time


exercise_name_schema = ResponseSchema(name="Exercise_Name",
                             description="What is the name of the exercise?")

response_schemas = [exercise_name_schema]

output_parser= StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()



filtered_result["Exercise_Name"]=None

for i in range(len(filtered_result)):

    prompt= ChatPromptTemplate.from_template(template=output_template)

    ## Making a prompt by looping over each item_description one by one
    messages= prompt.format_messages(text=filtered_result['exercise_name_list'].iloc[i],
                                     format_instructions=format_instructions)

    time.sleep(8)
    response= llm_model(messages)

    ## Getting the response and Parsing it to a json dictionary
    parsed_response= output_parser.parse(response.content)
    

    ## Now store this dictionary output back to the Answer field
    filtered_result['Exercise_Name'].iloc[i]= parsed_response

    messages=""

In [35]:
filtered_result

,day_of_week,exercise_name_list,sets_reps,rest,URL,Thumbnail_URL,Exercise_Name
0,Friday,Barbell Bent Over Row,3 sets of 8-12 reps,60 seconds,https://www.youtube.com/watch?v=FsK19s8eGFs,https://i.ytimg.com/vi/FsK19s8eGFs/hqdefault.j...,Exercise_Name Barbell Bent Over Row dtype: ...
1,Friday,Pull Ups,3 sets of as many reps as possible,60 seconds,https://www.youtube.com/watch?v=PHdHnZcbsB8,https://i.ytimg.com/vi/PHdHnZcbsB8/hqdefault.j...,Exercise_Name Pull Ups dtype: object
2,Friday,Reverse Grip Lat Pulldown,3 sets of 10-15 reps,60 seconds,https://www.youtube.com/watch?v=SNiwpA13ZLU,https://i.ytimg.com/vi/SNiwpA13ZLU/hqdefault.j...,Exercise_Name Reverse Grip Lat Pulldown dty...
3,Friday,Dumbbell Bent Over Row,3 sets of 8-12 reps per arm,60 seconds,https://www.youtube.com/watch?v=6gvmcqr226U,https://i.ytimg.com/vi/6gvmcqr226U/hqdefault.j...,Exercise_Name Dumbbell Bent Over Row dtype:...
4,Friday,uble Arm Curl (Dumbbell Bicep Curl),3 sets of 10-15 reps,60 seconds,https://www.youtube.com/watch?v=MtXdEcW3Eog,https://i.ytimg.com/vi/MtXdEcW3Eog/hqdefault.j...,Exercise_Name Double Arm Curl (Dumbbell Bic...


In [36]:
filtered_result["Display_Name"] = None
for i in range(len(filtered_result)):
  output=filtered_result['Exercise_Name'].iloc[i]

  filtered_result['Display_Name'].iloc[i]= output.get('Exercise_Name')

In [37]:
filtered_result

,day_of_week,exercise_name_list,sets_reps,rest,URL,Thumbnail_URL,Exercise_Name,Display_Name
0,Friday,Barbell Bent Over Row,3 sets of 8-12 reps,60 seconds,https://www.youtube.com/watch?v=FsK19s8eGFs,https://i.ytimg.com/vi/FsK19s8eGFs/hqdefault.j...,Exercise_Name Barbell Bent Over Row dtype: ...,Barbell Bent Over Row
1,Friday,Pull Ups,3 sets of as many reps as possible,60 seconds,https://www.youtube.com/watch?v=PHdHnZcbsB8,https://i.ytimg.com/vi/PHdHnZcbsB8/hqdefault.j...,Exercise_Name Pull Ups dtype: object,Pull Ups
2,Friday,Reverse Grip Lat Pulldown,3 sets of 10-15 reps,60 seconds,https://www.youtube.com/watch?v=SNiwpA13ZLU,https://i.ytimg.com/vi/SNiwpA13ZLU/hqdefault.j...,Exercise_Name Reverse Grip Lat Pulldown dty...,Reverse Grip Lat Pulldown
3,Friday,Dumbbell Bent Over Row,3 sets of 8-12 reps per arm,60 seconds,https://www.youtube.com/watch?v=6gvmcqr226U,https://i.ytimg.com/vi/6gvmcqr226U/hqdefault.j...,Exercise_Name Dumbbell Bent Over Row dtype:...,Dumbbell Bent Over Row
4,Friday,uble Arm Curl (Dumbbell Bicep Curl),3 sets of 10-15 reps,60 seconds,https://www.youtube.com/watch?v=MtXdEcW3Eog,https://i.ytimg.com/vi/MtXdEcW3Eog/hqdefault.j...,Exercise_Name Double Arm Curl (Dumbbell Bic...,Double Arm Curl (Dumbbell Bicep Curl)


#### Creating & Sending the Whatsapp Message

In [38]:
import datetime

markdown_output = ""

# Get the day of the week today
day_of_week_today = datetime.datetime.now().strftime("%A")
markdown_output += "Exercises for " + str(day_of_week_today) + " : " + str(datetime.date.today()) + "\n\n"

# Iterate through the DataFrame and build markdown output
for idx, row in filtered_result.iterrows():
    display_name = row['Display_Name']
    thumbnail_url = row['Thumbnail_URL']
    url = row['URL']
    
    markdown_output += f"*{idx + 1}) {display_name}*\n"
    markdown_output += f"[PureGym Video Link] ({url})\n\n"

print(markdown_output)


Exercises for Friday : 2024-11-15

*1) Barbell Bent Over Row*
[PureGym Video Link] (https://www.youtube.com/watch?v=FsK19s8eGFs)

*2) Pull Ups*
[PureGym Video Link] (https://www.youtube.com/watch?v=PHdHnZcbsB8)

*3) Reverse Grip Lat Pulldown*
[PureGym Video Link] (https://www.youtube.com/watch?v=SNiwpA13ZLU)

*4) Dumbbell Bent Over Row*
[PureGym Video Link] (https://www.youtube.com/watch?v=6gvmcqr226U)

*5) Double Arm Curl (Dumbbell Bicep Curl)*
[PureGym Video Link] (https://www.youtube.com/watch?v=MtXdEcW3Eog)




In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

from twilio.rest import Client

account_sid=os.getenv("TWILIO_ACCOUNT_SID")
auth_token=os.getenv("TWILIO_AUTH_TOKEN")

client = Client(account_sid, auth_token)

# Sending a message
from_whatsapp_number = 'whatsapp:+14155238886'  # Twilio sandbox number
to_whatsapp_number = 'whatsapp:+44XXXXXXXXXX'

message = client.messages.create(
    body=markdown_output,
    from_=from_whatsapp_number,
    to=to_whatsapp_number
)

print("Message sent! SID:", message.sid)


Message sent! SID: SM59947df101826433f68496a7f5d86876
